In [20]:
from pyspark.sql import SparkSession

In [21]:
spark = SparkSession.builder \
    .appName("HELK Reader") \
    .master("spark://helk-spark-master:7077") \
    .enableHiveSupport() \
    .getOrCreate()

In [22]:
es_reader = (spark.read
    .format("org.elasticsearch.spark.sql")
    .option("inferSchema", "true")
    .option("es.read.field.as.array.include", "tags")
    .option("es.nodes","helk-elasticsearch:9200")
    .option("es.net.http.auth.user","elastic")
)

In [23]:
sysmon_df = es_reader.load("logs-endpoint-winevent-sysmon-*/")

In [24]:
sysmon_df.createOrReplaceTempView("sysmon_events")

In [28]:
DLL_Injection_EID_8_10 = spark.sql(
'''
SELECT 
    a.host_name,
    a.process_name,
    a.process_granted_access,
    a.process_id, 
    a.process_target_name,
    c.thread_new_id
FROM sysmon_events c
JOIN sysmon_events a
    ON a.process_guid = c.process_guid
    AND a.event_id = 10
    AND (a.process_granted_access & 40) == 40 -- 40 is decimal for 0x0028. The minimal privileges you need to access process handle
WHERE
    c.event_id = 8
    AND c.thread_start_function = "LoadLibraryA" OR c.thread_start_function = "LoadLibraryW"
'''
).show(100,False)

+------------------+--------------+----------------------+----------+-------------------+-------------+
|host_name         |process_name  |process_granted_access|process_id|process_target_name|thread_new_id|
+------------------+--------------+----------------------+----------+-------------------+-------------+
|hr001.shire.com   |powershell.exe|2047999               |5452      |notepad.exe        |7940         |
|win10.marvel.local|injectdll.exe |2097151               |9028      |notepad.exe        |2216         |
|win10.marvel.local|injectdll.exe |2097151               |8220      |notepad.exe        |7524         |
|win10.marvel.local|injectdll.exe |2097151               |9204      |notepad.exe        |7972         |
|win10.marvel.local|injectdll.exe |2097151               |6440      |notepad.exe        |8492         |
|win10.marvel.local|injectdll.exe |2097151               |868       |notepad.exe        |8580         |
|win10.marvel.local|injectdll.exe |2097151               |2556  